In [8]:
# battle_system RPGシミュレーター - 戦闘システム
# 
# このノートブックでは、RPGシミュレーターの戦闘システムを実装します。
# 
# ## 目的
# - ターン制バトルシステムの実装
# - 戦闘結果の記録
# - 単純な敵AIの実装
# - 戦闘勝利後の報酬計算

import random
import time
from typing import Dict, List, Optional, Tuple
from IPython.display import clear_output
# 画面表示に必要なライブラリをインポート(3/19 画面作成で追加）
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
# インポート部分に追加（notebooks/battle_system.ipynb内）
import os
import matplotlib.image as mpimg


# characters.ipynbの内容を読み込む
%run ./notebooks/characters.ipynb

# ## バトルログクラス

class BattleLog:
    """戦闘ログを記録するクラス"""
    
    def __init__(self):
        """戦闘ログの初期化"""
        self.log = []
        self.turn_count = 0
        self.battle_result = None
        self.start_time = None
        self.end_time = None
    
    def start_battle(self, player_team: List[PlayerCharacter], enemy_team: List[Enemy]) -> None:
        """戦闘開始を記録
        
        Args:
            player_team: プレイヤーチーム
            enemy_team: 敵チーム
        """
        self.start_time = time.time()
        self.log.append({
            'turn': 0,
            'event': 'battle_start',
            'player_team': [p.name for p in player_team],
            'enemy_team': [e.name for e in enemy_team]
        })
    
    def end_battle(self, result: str, player_team: List[PlayerCharacter], enemy_team: List[Enemy]) -> None:
        """戦闘終了を記録
        
        Args:
            result: 戦闘結果（'victory', 'defeat', 'draw'）
            player_team: プレイヤーチーム
            enemy_team: 敵チーム
        """
        self.end_time = time.time()
        self.battle_result = result
        self.log.append({
            'turn': self.turn_count,
            'event': 'battle_end',
            'result': result,
            'player_team_status': [p.status() for p in player_team],
            'enemy_team_status': [e.status() for e in enemy_team],
            'duration': self.end_time - self.start_time
        })
    
    def new_turn(self) -> None:
        """新しいターンの開始を記録"""
        self.turn_count += 1
        self.log.append({
            'turn': self.turn_count,
            'event': 'turn_start'
        })
    
    def add_action(self, actor: Character, action_type: str, target: Optional[Character] = None,
                   damage: int = 0, healing: int = 0, mp_used: int = 0, skill_name: str = "") -> None:
        """行動を記録
        
        Args:
            actor: 行動者
            action_type: 行動タイプ（'attack', 'skill', 'item', 'defend', 'flee'）
            target: 行動対象（Optional）
            damage: 与えたダメージ
            healing: 回復量
            mp_used: 使用したMP
            skill_name: スキル名（スキル使用時）
        """
        action_log = {
            'turn': self.turn_count,
            'event': 'action',
            'actor': actor.name,
            'action_type': action_type
        }
        
        if target:
            action_log['target'] = target.name
            action_log['target_hp_before'] = target.current_hp + damage - healing
            action_log['target_hp_after'] = target.current_hp
        
        if damage > 0:
            action_log['damage'] = damage
        
        if healing > 0:
            action_log['healing'] = healing
        
        if mp_used > 0:
            action_log['mp_used'] = mp_used
        
        if skill_name:
            action_log['skill_name'] = skill_name
        
        self.log.append(action_log)
    
    def add_reward(self, exp_gained: int, items_gained: List[str] = None) -> None:
        """報酬の獲得を記録
        
        Args:
            exp_gained: 獲得した経験値
            items_gained: 獲得したアイテムリスト（Optional）
        """
        reward_log = {
            'turn': self.turn_count,
            'event': 'reward',
            'exp_gained': exp_gained
        }
        
        if items_gained:
            reward_log['items_gained'] = items_gained
        
        self.log.append(reward_log)
    
    def add_level_up(self, character: PlayerCharacter, old_level: int, new_level: int) -> None:
        """レベルアップを記録
        
        Args:
            character: レベルアップしたキャラクター
            old_level: 旧レベル
            new_level: 新レベル
        """
        self.log.append({
            'turn': self.turn_count,
            'event': 'level_up',
            'character': character.name,
            'old_level': old_level,
            'new_level': new_level,
            'status': character.status()
        })
    
    def print_battle_summary(self) -> None:
        """戦闘の要約を表示"""
        print(f"===== 戦闘サマリー =====")
        print(f"ターン数: {self.turn_count}")
        print(f"結果: {self.battle_result}")
        print(f"所要時間: {self.end_time - self.start_time:.2f}秒")
        
        # 戦闘開始時のチーム情報
        start_log = self.log[0]
        print(f"\nプレイヤーチーム: {', '.join(start_log['player_team'])}")
        print(f"敵チーム: {', '.join(start_log['enemy_team'])}")
        
        # ダメージと回復の合計を計算
        total_damage_dealt = sum(log.get('damage', 0) for log in self.log if log.get('event') == 'action' and 'damage' in log)
        total_healing = sum(log.get('healing', 0) for log in self.log if log.get('event') == 'action' and 'healing' in log)
        
        print(f"\n総ダメージ: {total_damage_dealt}")
        print(f"総回復量: {total_healing}")
        
        # 報酬情報
        rewards = [log for log in self.log if log.get('event') == 'reward']
        if rewards:
            total_exp = sum(reward.get('exp_gained', 0) for reward in rewards)
            print(f"\n獲得経験値: {total_exp}")
            
            # アイテム報酬
            all_items = []
            for reward in rewards:
                if 'items_gained' in reward:
                    all_items.extend(reward['items_gained'])
            
            if all_items:
                print(f"獲得アイテム: {', '.join(all_items)}")
        
        # レベルアップ情報
        level_ups = [log for log in self.log if log.get('event') == 'level_up']
        if level_ups:
            print(f"\nレベルアップしたキャラクター:")
            for level_up in level_ups:
                print(f"  {level_up['character']}: Lv.{level_up['old_level']} → Lv.{level_up['new_level']}")
# battle_system.ipynb内に追加
def load_image(image_path, default_draw_func=None, *args, **kwargs):
    """画像ファイルを読み込む、存在しない場合はデフォルト描画関数を実行
    
    Args:
        image_path: 画像ファイルのパス
        default_draw_func: デフォルトの描画関数（画像が読めない場合に使用）
        *args, **kwargs: デフォルト関数に渡す引数
        
    Returns:
        読み込んだ画像またはNone
    """
    try:
        if os.path.exists(image_path):
            return mpimg.imread(image_path)
        else:
            print(f"画像ファイルが見つかりません: {image_path}")
            if default_draw_func:
                default_draw_func(*args, **kwargs)
            return None
    except Exception as e:
        print(f"画像の読み込みに失敗しました: {e}")
        if default_draw_func:
            default_draw_func(*args, **kwargs)
        return None

def display_character_image(ax, character, x, y, width, height):
    """キャラクターの画像を表示する
    
    Args:
        ax: matplotlib axes
        character: キャラクターオブジェクト
        x, y: 描画位置
        width, height: サイズ
    """
    # キャラクター名から画像ファイル名を作成
    character_type = "enemies" if isinstance(character, Enemy) else "characters"
    image_name = character.name.lower().replace(" ", "_") + ".png"
    image_path = os.path.join(image_dir, character_type, image_name)
    
    # 画像の読み込みと表示
    img = load_image(image_path)
    if img is not None:
        ax.imshow(img, extent=[x, x+width, y, y+height], zorder=5)
        return True
    
    # 画像が読み込めなかった場合、スライムなら描画関数を使用
    if isinstance(character, Enemy):
        if "スライム" in character.name:
            draw_slime(ax, x, y, size=height)
            return True
        elif "オオカミ" in character.name:
            draw_wolf(ax, x, y, size=height)
            return True
        elif "ゴーレム" in character.name:
            draw_golem(ax, x, y, size=height)
            return True
    
    # それ以外はデフォルトの矩形を表示
    color = 'red' if isinstance(character, Enemy) else 'blue'
    ax.add_patch(Rectangle((x, y), width, height, facecolor=color))
    return False

def draw_slime(ax, x, y, size=10, color='#6495ED'):
    """スライムのグラフィックを描画する関数
    
    Args:
        ax: matplotlib axes
        x, y: 描画位置の座標
        size: スライムのサイズ
        color: スライムの色
    """
    import matplotlib.path as mpath
    import matplotlib.patches as mpatches
    
    # スライムの本体（楕円形）
    body_width = size * 1.2
    body_height = size * 0.8
    body = mpatches.Ellipse((x + size/2, y + size/2), 
                           body_width, body_height, 
                           facecolor=color, alpha=0.9, 
                           edgecolor='black', linewidth=1)
    ax.add_patch(body)
    
    # スライムの下部（波状の形）
    bottom_y = y + size/2 - body_height/2 + size*0.1
    wave_path = mpath.Path
    path_data = [
        (wave_path.MOVETO, (x, bottom_y)),
    ]
    
    # 波状の下部を作成
    num_waves = 5
    wave_width = body_width / num_waves
    wave_height = size * 0.15
    
    for i in range(num_waves + 1):
        wave_x = x + i * wave_width
        if i % 2 == 0:
            path_data.append(
                (wave_path.CURVE3, (wave_x, bottom_y - wave_height))
            )
        else:
            path_data.append(
                (wave_path.CURVE3, (wave_x, bottom_y))
            )
    
    path_data.append(
        (wave_path.CLOSEPOLY, (x, bottom_y))
    )
    
    codes, verts = zip(*path_data)
    path = mpath.Path(verts, codes)
    patch = mpatches.PathPatch(path, facecolor=color, alpha=0.8, 
                              edgecolor='black', linewidth=1)
    ax.add_patch(patch)
    
    # スライムの目（2つの黒い点）
    eye_size = size * 0.15
    eye_y = y + size*0.6
    # 左目
    left_eye = mpatches.Circle((x + size*0.35, eye_y), eye_size, 
                              facecolor='black', zorder=10)
    # 右目
    right_eye = mpatches.Circle((x + size*0.65, eye_y), eye_size, 
                               facecolor='black', zorder=10)
    
    ax.add_patch(left_eye)
    ax.add_patch(right_eye)
    
    # スライムのハイライト（白い楕円）
    highlight = mpatches.Ellipse((x + size*0.35, y + size*0.8), 
                                size*0.25, size*0.15, 
                                facecolor='white', alpha=0.7, zorder=5)
    ax.add_patch(highlight)
# draw_slime関数の後に追加
def draw_wolf(ax, x, y, size=10, color='#8B4513'):
    """オオカミのグラフィックを描画する関数
    
    Args:
        ax: matplotlib axes
        x, y: 描画位置の座標
        size: オオカミのサイズ
        color: オオカミの基本色
    """
    import matplotlib.path as mpath
    import matplotlib.patches as mpatches
    
    # オオカミの胴体（楕円形）
    body_width = size * 1.4
    body_height = size * 0.7
    body = mpatches.Ellipse((x + size/2, y + size/2), 
                           body_width, body_height, 
                           facecolor=color, alpha=0.9, 
                           edgecolor='black', linewidth=1)
    ax.add_patch(body)
    
    # 頭部
    head_size = size * 0.6
    head_x = x + size - head_size/2
    head_y = y + size/2 + body_height/4
    head = mpatches.Ellipse((head_x, head_y), 
                           head_size, head_size * 0.8, 
                           facecolor=color, alpha=0.9, 
                           edgecolor='black', linewidth=1)
    ax.add_patch(head)
    
    # 尻尾
    tail_path = mpath.Path
    path_data = [
        (tail_path.MOVETO, (x, y + size/2)),
        (tail_path.CURVE3, (x - size*0.3, y + size*0.7)),
        (tail_path.CURVE3, (x - size*0.4, y + size*0.9)),
    ]
    
    codes, verts = zip(*path_data)
    path = mpath.Path(verts, codes)
    patch = mpatches.PathPatch(path, facecolor='none', 
                              edgecolor='black', linewidth=2)
    ax.add_patch(patch)
    
    # 目
    eye_size = size * 0.1
    eye_x = head_x + head_size/4
    eye_y = head_y + head_size/6
    eye = mpatches.Circle((eye_x, eye_y), eye_size, 
                        facecolor='black', zorder=10)
    ax.add_patch(eye)
    
    # 耳
    ear_size = size * 0.2
    ear1_x = head_x - head_size/8
    ear1_y = head_y + head_size/2
    ear1 = mpatches.RegularPolygon((ear1_x, ear1_y), 3, 
                                 radius=ear_size, 
                                 orientation=0, 
                                 facecolor=color, edgecolor='black')
    
    ear2_x = head_x + head_size/4
    ear2_y = head_y + head_size/2
    ear2 = mpatches.RegularPolygon((ear2_x, ear2_y), 3, 
                                 radius=ear_size, 
                                 orientation=0, 
                                 facecolor=color, edgecolor='black')
    
    ax.add_patch(ear1)
    ax.add_patch(ear2)
def draw_golem(ax, x, y, size=10, color='#A9A9A9'):
    """ゴーレムのグラフィックを描画する関数
    
    Args:
        ax: matplotlib axes
        x, y: 描画位置の座標
        size: ゴーレムのサイズ
        color: ゴーレムの基本色
    """
    import matplotlib.patches as mpatches
    
    # 胴体（四角形）
    body_width = size * 0.8
    body_height = size * 1.0
    body = mpatches.Rectangle((x + size*0.1, y), 
                            body_width, body_height, 
                            facecolor=color, edgecolor='black', 
                            linewidth=2, alpha=0.9)
    ax.add_patch(body)
    
    # 頭
    head_size = size * 0.6
    head_x = x + size/2 - head_size/2
    head_y = y + body_height
    head = mpatches.Rectangle((head_x, head_y), 
                            head_size, head_size * 0.8, 
                            facecolor=color, edgecolor='black', 
                            linewidth=2, alpha=0.9)
    ax.add_patch(head)
    
    # 目（赤い光）
    eye_size = size * 0.1
    eye_spacing = size * 0.25
    
    left_eye = mpatches.Circle((head_x + head_size/2 - eye_spacing/2, head_y + head_size*0.5), 
                             eye_size, facecolor='red', alpha=0.8)
    right_eye = mpatches.Circle((head_x + head_size/2 + eye_spacing/2, head_y + head_size*0.5), 
                              eye_size, facecolor='red', alpha=0.8)
    
    ax.add_patch(left_eye)
    ax.add_patch(right_eye)
    
    # 腕
    arm_width = size * 0.25
    arm_height = size * 0.7
    
    # 左腕
    left_arm = mpatches.Rectangle((x - arm_width/2, y + size*0.3), 
                                arm_width, arm_height, 
                                facecolor=color, edgecolor='black', 
                                linewidth=1, alpha=0.9)
    
    # 右腕
    right_arm = mpatches.Rectangle((x + size - arm_width/2, y + size*0.3), 
                                 arm_width, arm_height, 
                                 facecolor=color, edgecolor='black', 
                                 linewidth=1, alpha=0.9)
    
    ax.add_patch(left_arm)
    ax.add_patch(right_arm)
    
    # 岩の模様（ひび割れ）
    crack_x = x + size*0.3
    crack_y = y + size*0.5
    ax.plot([crack_x, crack_x + size*0.4], [crack_y, crack_y + size*0.3], 
          color='black', linewidth=1, alpha=0.6)
    
    crack_x = x + size*0.5
    crack_y = y + size*0.2
    ax.plot([crack_x, crack_x - size*0.2], [crack_y, crack_y + size*0.4], 
          color='black', linewidth=1, alpha=0.6)
# 2025.3.19 画面描画関数
def display_battle_scene(player, enemy, turn=1, message="バトル開始！"):
    """戦闘画面を表示する関数
    
    Args:
        player: プレイヤーキャラクター
        enemy: 敵キャラクター
        turn: 現在のターン数
        message: 表示するメッセージ
    """
    # 日本語フォント設定
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['IPAGothic', 'Hiragino Sans', 'Meiryo', 'Yu Gothic', 'MS Gothic', 'Noto Sans CJK JP']
    
    plt.figure(figsize=(10, 6))
    ax = plt.gca()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 60)
    
    # 背景色
    ax.add_patch(Rectangle((0, 0), 100, 60, facecolor='#e6e6fa'))
    
    # プレイヤーキャラクター
    display_character_image(ax, player, 20, 15, 10, 15)
    
    # 敵キャラクター
    if enemy:
        # display_character_imageを使用して敵の画像を表示
        display_character_image(ax, enemy, 70, 15, 10, 10)
    
    # プレイヤーHPバー
    max_hp_width = 20
    hp_ratio = max(0, player.current_hp / player.max_hp)
    hp_width = max_hp_width * hp_ratio
    ax.add_patch(Rectangle((10, 40), max_hp_width, 3, facecolor='#cccccc'))
    ax.add_patch(Rectangle((10, 40), hp_width, 3, facecolor='green'))
    
    # 敵HPバー
    if enemy:
        enemy_hp_ratio = max(0, enemy.current_hp / enemy.max_hp)
        enemy_hp_width = max_hp_width * enemy_hp_ratio
        ax.add_patch(Rectangle((70, 40), max_hp_width, 3, facecolor='#cccccc'))
        ax.add_patch(Rectangle((70, 40), enemy_hp_width, 3, facecolor='red'))
    
    # MPバー（プレイヤー）
    mp_ratio = player.current_mp / player.max_mp
    mp_width = max_hp_width * mp_ratio
    ax.add_patch(Rectangle((10, 37), max_hp_width, 2, facecolor='#cccccc'))
    ax.add_patch(Rectangle((10, 37), mp_width, 2, facecolor='blue'))
    
    # テキスト（日本語対応）
    ax.text(10, 45, f"{player.name} (Lv.{player.level})")
    ax.text(10, 42, f"HP: {player.current_hp}/{player.max_hp}")
    ax.text(10, 34, f"MP: {player.current_mp}/{player.max_mp}")
    
    if enemy:
        ax.text(70, 45, f"{enemy.name} (Lv.{enemy.level})")
        ax.text(70, 42, f"HP: {enemy.current_hp}/{enemy.max_hp}")
    
    # タイトルとターン情報
    ax.text(50, 55, "RPGバトル", fontsize=14, ha='center')
    ax.text(50, 52, f"ターン {turn}", ha='center')
    
    # メッセージ領域
    ax.add_patch(Rectangle((10, 5), 80, 8, facecolor='white', alpha=0.7))
    ax.text(50, 9, message, ha='center')
    
    # コマンド領域
    commands = ["攻撃", "スキル", "アイテム", "逃げる"]
    for i, cmd in enumerate(commands):
        x = 15 + i * 20
        ax.add_patch(Rectangle((x, 1), 15, 3, facecolor='#dddddd'))
        ax.text(x + 7.5, 2.5, cmd, ha='center', va='center')
    
    # 軸を非表示
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()    
def display_attack_effect(player, enemy, damage, effect_type="slash"):
    """攻撃エフェクトを表示
    
    Args:
        player: 攻撃するキャラクター
        enemy: 攻撃対象
        damage: ダメージ量
        effect_type: エフェクトタイプ
    """
    # 日本語フォント設定
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['IPAGothic', 'Hiragino Sans', 'Meiryo', 'Yu Gothic', 'MS Gothic', 'Noto Sans CJK JP']
    
    plt.figure(figsize=(10, 6))
    ax = plt.gca()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 60)
    
    # 背景画像
    bg_path = os.path.join(background_dir, "battle_bg.png")
    bg_img = load_image(bg_path)
    if bg_img is not None:
        ax.imshow(bg_img, extent=[0, 100, 0, 60], aspect='auto', zorder=0)
    else:
        ax.add_patch(Rectangle((0, 0), 100, 60, facecolor='#e6e6fa', zorder=0))
    
    # 攻撃するキャラクター
    display_character_image(ax, player, 15, 15, 15, 15)
    
    # 攻撃対象のキャラクター
    display_character_image(ax, enemy, 70, 15, 15, 15)
    
    # エフェクト画像の表示
    effect_image_path = os.path.join(image_dir, 'effects', f"{effect_type}.png")
    effect_img = load_image(effect_image_path)
    
    if effect_img is not None:
        # エフェクト画像を敵の位置に表示
        ax.imshow(effect_img, extent=[65, 85, 10, 30], zorder=10)
    else:
        # エフェクト画像がない場合は簡易的なエフェクト
        if effect_type == "slash":
            # 斬撃エフェクト
            ax.plot([65, 85], [25, 15], 'r-', linewidth=2, zorder=10)
            ax.plot([65, 85], [15, 25], 'r-', linewidth=2, zorder=10)
        elif effect_type == "magic":
            # 魔法エフェクト
            circle = plt.Circle((75, 20), 8, color='purple', alpha=0.6, zorder=10)
            ax.add_artist(circle)
    
    # ダメージ表示
    ax.text(75, 35, f"{damage}", color='red', fontsize=18, 
            fontweight='bold', ha='center', va='center', zorder=11)
    
    # 軸を非表示
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
# ## 戦闘システムクラス

class BattleSystem:
    """RPGの戦闘システムを管理するクラス"""
    
    def __init__(self, player_team: List[PlayerCharacter], enemy_team: List[Enemy],
                 show_detailed_log: bool = True, visual_mode: bool = False):
        """戦闘システムの初期化
        
        Args:
            player_team: プレイヤーキャラクターのリスト
            enemy_team: 敵キャラクターのリスト
            show_detailed_log: 詳細な戦闘ログを表示するかどうか
            visual_mode: 視覚的表示モードを使用するかどうか
        """
        self.player_team = player_team
        self.enemy_team = enemy_team
        self.battle_log = BattleLog()
        self.show_detailed_log = show_detailed_log
        self.visual_mode = visual_mode
        self.current_turn = 0
    
    def start_battle(self) -> str:
        """戦闘を開始し、結果を返す
        
        Returns:
            戦闘結果（'victory', 'defeat', 'draw'）
        """
        # 戦闘開始のログを記録
        self.battle_log.start_battle(self.player_team, self.enemy_team)
        
        # キャラクターの状態をリセット
        for character in self.player_team + self.enemy_team:
            character.reset()
        
        # 戦闘開始メッセージ
        enemy_names = [enemy.name for enemy in self.enemy_team]
        if len(self.enemy_team) == 1 and "スライム" in self.enemy_team[0].name:
            self._print_battle_message(f"{self.enemy_team[0].name}がぷるぷると現れた！")
        else:
            self._print_battle_message(f"{'、'.join(enemy_names)}が現れた！")
        
        # 視覚モードが有効な場合、戦闘開始画面を表示
        if self.visual_mode and self.player_team and self.enemy_team:
            display_battle_scene(
                self.player_team[0], 
                self.enemy_team[0], 
                turn=0, 
                message="バトル開始！"
            )
            time.sleep(1.0)
            clear_output(wait=True)
        
        # メインバトルループ
        while True:
            # 新しいターンの開始
            self.current_turn += 1
            self.battle_log.new_turn()
            self._print_battle_message(f"\n===== ターン {self.current_turn} =====")
            
            # 全キャラクターのリストを作成し、素早さに基づいて行動順を決定
            all_characters = self.player_team + self.enemy_team
            turn_order = sorted(all_characters, key=lambda x: x.speed, reverse=True)
            
            # 各キャラクターの行動
            for character in turn_order:
                if not character.is_alive:
                    continue
                
                # プレイヤーか敵かによって行動を決定
                if character in self.player_team:
                    self._player_action(character)
                else:  # 敵の行動
                    self._enemy_action(character)
                
                # 視覚モードが有効な場合、アクション後の状態を表示
                if self.visual_mode and self.player_team and self.enemy_team:
                    active_player = next((p for p in self.player_team if p.is_alive), None)
                    active_enemy = next((e for e in self.enemy_team if e.is_alive), None)
                    
                    if active_player and active_enemy:
                        # 最後の行動に基づいてメッセージを設定
                        last_log = self.battle_log.log[-1] if self.battle_log.log else None
                        message = "バトル中..."
                        
                        if last_log and last_log.get('event') == 'action':
                            if 'damage' in last_log:
                                message = f"{last_log['target']}に{last_log['damage']}のダメージ！"
                            elif 'healing' in last_log:
                                message = f"{last_log['target']}が{last_log['healing']}回復！"
                        
                        display_battle_scene(
                            active_player,
                            active_enemy,
                            turn=self.current_turn,
                            message=message
                        )
                        time.sleep(0.8)
                        clear_output(wait=True)
                
                # 戦闘終了条件をチェック
                battle_result = self._check_battle_end()
                if battle_result != 'continue':
                    # 戦闘終了のログを記録
                    self.battle_log.end_battle(battle_result, self.player_team, self.enemy_team)
                    
                    if battle_result == 'victory':
                        self._handle_victory()
                        
                        # 勝利画面の表示（視覚モード）
                        if self.visual_mode and self.player_team:
                            active_player = next((p for p in self.player_team if p.is_alive), None)
                            if active_player:
                                display_battle_scene(
                                    active_player, 
                                    self.enemy_team[0] if self.enemy_team else None,
                                    turn=self.current_turn,
                                    message="戦闘に勝利した！"
                                )
                                time.sleep(1.5)
                                clear_output(wait=True)
                            
                    elif battle_result == 'defeat':
                        self._print_battle_message("プレイヤーチームは全滅した...")
                        
                        # 敗北画面の表示（視覚モード）
                        if self.visual_mode and self.enemy_team:
                            active_enemy = next((e for e in self.enemy_team if e.is_alive), None)
                            if active_enemy:
                                display_battle_scene(
                                    self.player_team[0] if self.player_team else None,
                                    active_enemy,
                                    turn=self.current_turn,
                                    message="全滅してしまった..."
                                )
                                time.sleep(1.5)
                                clear_output(wait=True)
                    
                    # 戦闘のサマリーを表示
                    if self.show_detailed_log:
                        self.battle_log.print_battle_summary()
                    
                    return battle_result
            
            # ターン終了時の状態表示
            self._display_battle_status()
            
            # 視覚モードのターン終了表示
            if self.visual_mode and self.player_team and self.enemy_team:
                active_player = next((p for p in self.player_team if p.is_alive), None)
                active_enemy = next((e for e in self.enemy_team if e.is_alive), None)
                
                if active_player and active_enemy:
                    display_battle_scene(
                        active_player,
                        active_enemy,
                        turn=self.current_turn,
                        message=f"ターン {self.current_turn} 終了"
                    )
                    time.sleep(1.0)
                    clear_output(wait=True)
    
    def _player_action(self, character: PlayerCharacter) -> None:
        """プレイヤーキャラクターの行動を処理
        
        Args:
            character: 行動するプレイヤーキャラクター
        """
        # シンプルな実装：最初の生存している敵に攻撃
        alive_enemies = [enemy for enemy in self.enemy_team if enemy.is_alive]
        
        if not alive_enemies:
            return
        
        target = alive_enemies[0]
        
        # スライムに対する特別なメッセージ
        if "スライム" in target.name:
            self._print_battle_message(f"{character.name}の攻撃！ スライムに剣を振り下ろした！")
        else:
            self._print_battle_message(f"{character.name}の攻撃！")
        
        damage = character.attack(target)
        self._print_battle_message(f"{target.name}に{damage}のダメージ！")
        
        # 視覚モードが有効な場合、攻撃エフェクトを表示
        if self.visual_mode:
            display_attack_effect(character, target, damage, "slash")
            time.sleep(0.8)
            clear_output(wait=True)
        
        # 行動を記録
        self.battle_log.add_action(character, 'attack', target, damage=damage)
        
        # 敵が倒れた場合
        if not target.is_alive:
            if "スライム" in target.name:
                self._print_battle_message(f"{target.name}はぷちゅんと弾けて消えた！")
            else:
                self._print_battle_message(f"{target.name}を倒した！")
    def _enemy_action(self, enemy: Enemy) -> None:
        """敵キャラクターの行動を処理
        
        Args:
            enemy: 行動する敵キャラクター
        """
        # 行動対象の候補
        alive_players = [player for player in self.player_team if player.is_alive]
        
        if not alive_players:
            return
        
        # 敵AIで行動を決定
        action = enemy.get_action(alive_players)
        
        if action['type'] == 'attack':
            target = action['target']
            
            # スライムの特別な攻撃メッセージ
            if "スライム" in enemy.name:
                self._print_battle_message(f"{enemy.name}がぷるぷると体を揺らした！")
            else:
                self._print_battle_message(f"{enemy.name}の攻撃！")
            
            damage = enemy.attack(target)
            self._print_battle_message(f"{target.name}に{damage}のダメージ！")
            
            # 行動を記録
            self.battle_log.add_action(enemy, 'attack', target, damage=damage)
            
            # プレイヤーが倒れた場合
            if not target.is_alive:
                self._print_battle_message(f"{target.name}は倒れた！")
    def _check_battle_end(self) -> str:
        """戦闘終了条件をチェック
        
        Returns:
            戦闘状態（'victory', 'defeat', 'draw', 'continue'）
        """
        # 全ての敵が倒れているか
        all_enemies_defeated = all(not enemy.is_alive for enemy in self.enemy_team)
        
        # 全てのプレイヤーが倒れているか
        all_players_defeated = all(not player.is_alive for player in self.player_team)
        
        if all_enemies_defeated and all_players_defeated:
            return 'draw'
        elif all_enemies_defeated:
            return 'victory'
        elif all_players_defeated:
            return 'defeat'
        else:
            return 'continue'
    
    def _handle_victory(self) -> None:
        """戦闘勝利時の処理"""
        self._print_battle_message("\n戦闘に勝利した！")
        
        # 経験値報酬の計算
        total_exp = sum(enemy.exp_reward for enemy in self.enemy_team)
        exp_per_player = total_exp // len(self.player_team)
        
        self._print_battle_message(f"各プレイヤーに{exp_per_player}の経験値を獲得！")
        
        # 経験値の獲得を記録
        self.battle_log.add_reward(total_exp)
        
        # 各プレイヤーに経験値を分配
        for player in self.player_team:
            if player.is_alive:
                old_level = player.level
                leveled_up = player.gain_exp(exp_per_player)
                
                if leveled_up:
                    self._print_battle_message(f"{player.name}はレベル{old_level}から{player.level}にレベルアップした！")
                    self.battle_log.add_level_up(player, old_level, player.level)
    
    def _display_battle_status(self) -> None:
        """戦闘の現在の状態を表示"""
        self._print_battle_message("\n----- 現在の状態 -----")
        
        # プレイヤーチームの状態
        self._print_battle_message("[プレイヤーチーム]")
        for player in self.player_team:
            status = "戦闘不能" if not player.is_alive else f"HP:{player.current_hp}/{player.max_hp} MP:{player.current_mp}/{player.max_mp}"
            self._print_battle_message(f"{player.name}: {status}")
        
        # 敵チームの状態
        self._print_battle_message("\n[敵チーム]")
        for enemy in self.enemy_team:
            if enemy.is_alive:
                self._print_battle_message(f"{enemy.name}: HP:{enemy.current_hp}/{enemy.max_hp}")
    
    def _print_battle_message(self, message: str) -> None:
        """戦闘メッセージを表示
        
        Args:
            message: 表示するメッセージ
        """
        if self.show_detailed_log:
            print(message)


# ## 戦闘シミュレーション関数

def simulate_battle(player_team: List[PlayerCharacter], enemy_team: List[Enemy],
                    show_detailed_log: bool = True, visual_mode: bool = False) -> Tuple[str, BattleLog]:
    """戦闘をシミュレートする
    
    Args:
        player_team: プレイヤーチーム
        enemy_team: 敵チーム
        show_detailed_log: 詳細ログを表示するかどうか
        visual_mode: 視覚的表示モードを使用するかどうか
        
    Returns:
        戦闘結果と戦闘ログのタプル
    """
    battle_system = BattleSystem(player_team, enemy_team, show_detailed_log, visual_mode)
    result = battle_system.start_battle()
    return result, battle_system.battle_log

def simulate_multiple_battles(player_team: List[PlayerCharacter], enemy_team: List[Enemy],
                             num_battles: int = 10, show_detailed_log: bool = False) -> Dict:
    """複数回の戦闘をシミュレートして結果を集計
    
    Args:
        player_team: プレイヤーチーム
        enemy_team: 敵チーム
        num_battles: シミュレーション回数
        show_detailed_log: 詳細ログを表示するかどうか
        
    Returns:
        シミュレーション結果の統計情報
    """
    results = {
        'victory': 0,
        'defeat': 0,
        'draw': 0,
        'total_turns': 0,
        'total_time': 0.0,
        'logs': []
    }
    
    for i in range(num_battles):
        # 戦闘前にキャラクターの状態をリセット
        player_copy = []
        enemy_copy = []
        
        # ディープコピーでキャラクターを複製
        for player in player_team:
            new_player = PlayerCharacter(
                name=player.name,
                hp=player.max_hp,
                mp=player.max_mp,
                strength=player.strength,
                defense=player.defense,
                magic=player.magic,
                speed=player.speed,
                level=player.level,
                job=player.job
            )
            player_copy.append(new_player)
            
        for enemy in enemy_team:
            new_enemy = Enemy(
                name=enemy.name,
                hp=enemy.max_hp,
                mp=enemy.max_mp,
                strength=enemy.strength,
                defense=enemy.defense,
                magic=enemy.magic,
                speed=enemy.speed,
                level=enemy.level,
                exp_reward=enemy.exp_reward,
                enemy_type=enemy.enemy_type
            )
            enemy_copy.append(new_enemy)
        
        print(f"\nシミュレーション {i+1}/{num_battles}")
        result, battle_log = simulate_battle(player_copy, enemy_copy, show_detailed_log)
        
        # 結果をカウント
        results[result] += 1
        results['total_turns'] += battle_log.turn_count
        results['total_time'] += battle_log.end_time - battle_log.start_time
        results['logs'].append(battle_log.log)
    
    # 統計情報を計算
    results['victory_rate'] = results['victory'] / num_battles * 100
    results['average_turns'] = results['total_turns'] / num_battles
    results['average_time'] = results['total_time'] / num_battles
    
    return results

# ## サンプルバトルテスト

def test_sample_battle():
    """サンプルの戦闘をテストする"""
    # サンプルキャラクター作成
    hero = create_sample_character()
    slime = create_sample_enemy()
    
    # 戦闘実行
    print("===== サンプル戦闘テスト =====")
    result, log = simulate_battle([hero], [slime], show_detailed_log=True, visual_mode=False)
    
    print(f"\n戦闘結果: {result}")
    return result, log

# # サンプル戦闘テスト実行
# if __name__ == "__main__":
#     test_result, test_log = test_sample_battle()

Exception: File `'./notebooks/characters.ipynb'` not found.